In [3]:
import torch
import torch.nn as nn
import numpy as np
from board_funcs import get_sample_board, board_to_np

In [1]:
#Constant params
nc = 3   #number channels
nf = 32  #number feature detectors
board_size = 21
action_space_size = 5
agent_count = 4

#use GPU if available
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [9]:
class ShipNet(nn.Module):
    def __init__(self):
        super(ShipNet, self).__init__()
        
        #four convolutional layers
        self.conv = nn.Sequential(
            nn.Conv2d(nc, nf, 4, 1, 0), # out 18x18
            nn.BatchNorm2d(nf),
            nn.LeakyReLU(0.2, inplace=True), 

            nn.Conv2d(nf, nf, 3, 1, 0), # out 16x16
            nn.BatchNorm2d(nf),
            nn.LeakyReLU(0.2, inplace=True), 

            nn.Conv2d(nf, nf*2, 4, 2, 1), # out 8x8
            nn.BatchNorm2d(nf*2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(nf*2, nf*2, 4, 2, 1), # out 4x4
            nn.BatchNorm2d(nf*2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(nf*2, nf*4, 4, 2, 1), # out 2x2
            nn.BatchNorm2d(nf*2),
            nn.LeakyReLU(0.2, inplace=True)
        )

        #single linear layer
        self.linear = nn.Sequential(
            nn.Linear(nf*4, action_space_size)
        )
        
    def forward(self, x):
        #forward pass through the network
        x = self.conv(x)
        x = self.view(-1, ngf*4)
        return self.linear(x)

In [11]:
#create test batch of size 5 
bs = 5
boards = []
for i in range(5): 
    sample = get_sample_board(board_size, agent_count)
    sample_np = board_to_np(sample, nc)
    boards.append(sample_np)
batch = torch.tensor(boards)
batch.shape

torch.Size([5, 3, 21, 21])

In [12]:
#init network and run on test batch 
NN = ShipNet().to(device)